## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
import sys
sys.path.append('../')

# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Porto Santo,PT,2022-02-01 20:28:30,33.0720,-16.3415,64.31,82,20,21.85,few clouds
1,1,Kaitangata,NZ,2022-02-01 20:28:31,-46.2817,169.8464,61.41,91,100,3.65,overcast clouds
2,2,Vigrestad,NO,2022-02-01 20:28:31,58.5667,5.7000,44.28,80,32,30.94,scattered clouds
3,3,Bluff,NZ,2022-02-01 20:27:32,-46.6000,168.3333,70.81,74,100,13.04,overcast clouds
4,4,Naze,JP,2022-02-01 20:25:47,28.3667,129.4833,55.63,71,100,10.74,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Porto Santo,PT,2022-02-01 20:28:30,33.0720,-16.3415,64.31,82,20,21.85,few clouds
1,1,Kaitangata,NZ,2022-02-01 20:28:31,-46.2817,169.8464,61.41,91,100,3.65,overcast clouds
3,3,Bluff,NZ,2022-02-01 20:27:32,-46.6000,168.3333,70.81,74,100,13.04,overcast clouds
4,4,Naze,JP,2022-02-01 20:25:47,28.3667,129.4833,55.63,71,100,10.74,overcast clouds
14,14,Bredasdorp,ZA,2022-02-01 20:28:35,-34.5322,20.0403,75.60,76,3,3.06,clear sky
18,18,Phrae,TH,2022-02-01 20:28:37,18.0833,100.0000,62.67,64,85,1.03,overcast clouds
19,19,Port Lincoln,AU,2022-02-01 20:25:40,-34.7333,135.8667,60.66,56,52,17.87,broken clouds
25,25,Kensington,AU,2022-02-01 20:28:40,-33.9167,151.2167,72.72,93,75,18.41,light rain
26,26,San Jose,US,2022-02-01 20:24:11,37.3394,-121.8950,60.91,30,0,10.00,clear sky
30,30,Mataura,NZ,2022-02-01 20:28:41,-46.1927,168.8643,70.48,66,100,5.50,overcast clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        407
City           407
Country        393
Date           407
Lat            407
Lng            407
Max Temp       407
Humidity       407
Cloudiness     407
Wind Speed     407
Description    407
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()

preferred_cities_df.count()

City_ID        393
City           393
Country        393
Date           393
Lat            393
Lng            393
Max Temp       393
Humidity       393
Cloudiness     393
Wind Speed     393
Description    393
dtype: int64

In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Porto Santo,PT,64.31,few clouds,33.0720,-16.3415,
1,Kaitangata,NZ,61.41,overcast clouds,-46.2817,169.8464,
3,Bluff,NZ,70.81,overcast clouds,-46.6000,168.3333,
4,Naze,JP,55.63,overcast clouds,28.3667,129.4833,
14,Bredasdorp,ZA,75.60,clear sky,-34.5322,20.0403,
18,Phrae,TH,62.67,overcast clouds,18.0833,100.0000,
19,Port Lincoln,AU,60.66,broken clouds,-34.7333,135.8667,
25,Kensington,AU,72.72,light rain,-33.9167,151.2167,
26,San Jose,US,60.91,clear sky,37.3394,-121.8950,
30,Mataura,NZ,70.48,overcast clouds,-46.1927,168.8643,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")



    
    
    

    
   
    



    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [23]:
hotel_df.head() 

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Porto Santo,PT,64.31,few clouds,33.0720,-16.3415,Hotel Porto Santo & Spa
1,Kaitangata,NZ,61.41,overcast clouds,-46.2817,169.8464,Kaitangata Motor Camp
3,Bluff,NZ,70.81,overcast clouds,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
4,Naze,JP,55.63,overcast clouds,28.3667,129.4833,Shishime Hotel
14,Bredasdorp,ZA,75.60,clear sky,-34.5322,20.0403,Bredasdorp Country Manor


In [22]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()

In [17]:
# 8a. Create the output File (CSV)
output_data_file = "../Weather_Database/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
preferred_cities_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Weather: </dt><dd>{Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))